In [1]:
%pip install torch
%pip install torchsummary
%pip install -r /home/artificial-architecture/Desktop/TripoFT-adya/TripoSR/requirements.txt

You should consider upgrading via the '/home/artificial-architecture/Desktop/TripoFT-adya/envi/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/artificial-architecture/Desktop/TripoFT-adya/envi/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-oob0m3me
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-oob0m3me
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3aef8afa5f21b113afc4f4ea148baee850cbd472
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/home/artificial-architecture/Desktop/TripoFT-adya/envi/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.3.0+cu121
12.1


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import sys
import os

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR
from omegaconf import OmegaConf


In [5]:
import sys
import os
import torch
from omegaconf import OmegaConf

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR

# Load the configuration file
config_path = 'TripoSR/config.yaml'
cfg = OmegaConf.load(config_path)

# Manually set `tokenizer.num_channels` before initializing other components
cfg.backbone.in_channels = cfg.tokenizer.num_channels

# Initialize the TSR model with the loaded and updated configuration
model = TSR(cfg=cfg)

/home/artificial-architecture/Desktop/TripoFT-adya/envi/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/artificial-architecture/Desktop/TripoFT-adya/envi/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
# # Move model to GPU 1
# model = model.to("cuda:1")import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class VoxelGridDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.npz')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)
        voxel_grid = data['voxel_grid']

        # Convert to torch tensor
        voxel_grid = torch.tensor(voxel_grid, dtype=torch.float32)

        return voxel_grid

def custom_collate_fn(batch):
    # No need to pad since all voxel grids should be the same shape
    voxel_grids = torch.stack(batch)
    return voxel_grids

def move_to_gpu(batch):
    return batch.cuda()

def main():
    # Paths to your data directories
    train_dir = '/home/artificial-architecture/Desktop/TripoFT-adya/split_data/train'
    val_dir = '/home/artificial-architecture/Desktop/TripoFT-adya/split_data/val'
    test_dir = '/home/artificial-architecture/Desktop/TripoFT-adya/split_data/test'

    # Create dataset instances
    train_dataset = VoxelGridDataset(train_dir)
    val_dataset = VoxelGridDataset(val_dir)
    test_dataset = VoxelGridDataset(test_dir)

    # Create data loaders with custom collate function
    batch_size = 1  # Adjust as needed
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)

    # Example of iterating through the train_loader
    for voxel_grid in train_loader:
        print(voxel_grid.shape)
        # Move to GPU
        voxel_grid = move_to_gpu(voxel_grid)
        print(voxel_grid.shape)
        # Add your training code here
        break

if __name__ == "__main__":
    main()


# # Wrap the model with DataParallel to utilize GPU 1
# model = nn.DataParallel(model, device_ids=[1])


torch.Size([1, 1, 33, 19, 33])
torch.Size([1, 1, 33, 19, 33])


In [7]:
import os
import time  # To measure training time per epoch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.nn.parallel import DataParallel
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import gc

# Define a loss function using Chamfer Distance
class ChamferDistance(nn.Module):
    def forward(self, points1, points2):
        points1 = points1.float()
        points2 = points2.float()
        batch_size, num_points, _ = points1.size()
        dist1 = torch.cdist(points1, points2)
        dist2 = torch.cdist(points2, points1)
        loss1 = dist1.min(dim=2)[0].mean(dim=1)
        loss2 = dist2.min(dim=2)[0].mean(dim=1)
        loss = (loss1 + loss2).mean()
        return loss

loss_fn = ChamferDistance()

class ExampleModel(nn.Module):
    def __init__(self):
        super(ExampleModel, self).__init__()
        self.conv1 = nn.Conv3d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.fc1 = None  # Initialize fc1 as None
        self.fc2 = nn.Linear(256, 1024 * 3)  # Adjust to output a batch of 1024 points (each with 3 coordinates)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1)
        if self.fc1 is None:
            flattened_size = x.size(1)
            self.fc1 = nn.Linear(flattened_size, 256)
            self.fc1.to(x.device)  # Ensure the new layer is on the correct device
        x = self.fc1(x)
        x = self.fc2(x)
        x = x.view(-1, 1024, 3)  # Reshape to [batch_size, num_points, 3]
        return x

# Function to transform model output to point cloud
def transform_output_to_point_cloud(output):
    return output

# Wrapper function to handle model call with device
def call_model_with_device(model, vertices, device):
    return model(vertices.to(device))

# Function to calculate validation metrics
def validate(model, val_loader, loss_fn, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for voxel_grid in val_loader:
            voxel_grid = voxel_grid.to(device)
            voxel_grid_points = voxel_grid.view(voxel_grid.size(0), -1, 3)
            model_output = call_model_with_device(model, voxel_grid, device)
            outputs = transform_output_to_point_cloud(model_output)
            loss = loss_fn(outputs, voxel_grid_points)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    return avg_val_loss

# Function to calculate training accuracy
def calculate_accuracy(outputs, voxel_grid_points):
    chamfer_dist = ChamferDistance()
    outputs = outputs.float()
    voxel_grid_points = voxel_grid_points.float()
    accuracy = 1 / (1 + chamfer_dist(outputs, voxel_grid_points).item())
    return accuracy

# Training function with gradient accumulation, mixed precision, and early stopping
def train(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device, accumulation_steps=4, patience=3):
    scaler = GradScaler()  # For mixed precision
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0  # Initialize running accuracy

        print(f"Batch size: {train_loader.batch_size}")  # Print batch size at the beginning of the epoch

        for i, voxel_grid in enumerate(train_loader):
            voxel_grid = voxel_grid.to(device)
            voxel_grid_points = voxel_grid.view(voxel_grid.size(0), -1, 3)

            optimizer.zero_grad()
            with autocast():  # Mixed precision context
                model_output = call_model_with_device(model, voxel_grid, device)  # Call model with device argument
                outputs = transform_output_to_point_cloud(model_output)  # Transform to point cloud
                loss = loss_fn(outputs, voxel_grid_points)
                loss = loss / accumulation_steps  # Normalize the loss

            scaler.scale(loss).backward()  # Scaled loss backward

            if (i + 1) % accumulation_steps == 0:  # Perform step every accumulation_steps
                scaler.step(optimizer)  # Optimizer step
                scaler.update()  # Update scaler
                optimizer.zero_grad()

            running_loss += loss.item() * accumulation_steps  # Adjust running loss
            running_accuracy += calculate_accuracy(outputs, voxel_grid_points)  # Accumulate accuracy

        avg_train_loss = running_loss / len(train_loader)
        avg_train_accuracy = running_accuracy / len(train_loader)  # Average accuracy

        # Validation step
        avg_val_loss = validate(model, val_loader, loss_fn, device)

        # Gradient norm calculation
        total_norm = 0.0
        for p in model.parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** 0.5

        # Memory usage
        memory_allocated = torch.cuda.memory_allocated(device) / (1024 ** 3)  # Convert to GB
        memory_reserved = torch.cuda.memory_reserved(device) / (1024 ** 3)  # Convert to GB

        # Time per epoch
        epoch_time = time.time() - start_time

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, '
              f'Train Accuracy: {avg_train_accuracy:.4f}, '  # Print training accuracy
              f'Validation Loss: {avg_val_loss:.4f}, '
              f'Training Time: {epoch_time:.2f}s, '
              f'Gradient Norm: {total_norm:.4f}, '
              f'Memory Allocated: {memory_allocated:.2f}GB, '
              f'Memory Reserved: {memory_reserved:.2f}GB')

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 3  # Reset patience counter
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

        # Clear unused variables and cache
        del voxel_grid, outputs, loss, model_output
        torch.cuda.empty_cache()
        gc.collect()

class VoxelGridDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.npz')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)
        voxel_grid = data['voxel_grid']
        voxel_grid = torch.tensor(voxel_grid, dtype=torch.float32)
        return voxel_grid

def custom_collate_fn(batch):
    voxel_grids = torch.stack(batch)
    return voxel_grids

def move_to_gpu(batch):
    return batch.cuda()

def main():
    # Paths to your data directories
    train_dir = '/home/artificial-architecture/Desktop/TripoFT-adya/split_data/train'
    val_dir = '/home/artificial-architecture/Desktop/TripoFT-adya/split_data/val'
    test_dir = '/home/artificial-architecture/Desktop/TripoFT-adya/split_data/test'

    # Create dataset instances
    train_dataset = VoxelGridDataset(train_dir)
    val_dataset = VoxelGridDataset(val_dir)
    test_dataset = VoxelGridDataset(test_dir)

    # Create data loaders with custom collate function
    batch_size = 1  # Adjust as needed
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)

    # Move model to GPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = ExampleModel().to(device)

    # Wrap the model with DataParallel to utilize multiple GPUs
    model = nn.DataParallel(model, device_ids=[0, 1, 2, 3])

    # Print the model summary
    print("Model Summary:")
    summary(model, (1, 32, 32, 32))  # Assuming the input size is (1, 32, 32, 32), adjust as needed

    # Print all model parameters
    print("\nModel Parameters:")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {param.shape}")

    # Define an optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adjust learning rate as needed

    # Define the number of epochs
    num_epochs = 20  # Adjust as needed

    # Train the model with early stopping
    train(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device)

if __name__ == "__main__":
    main()

Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 32, 32, 32]           1,792
            Conv3d-2       [-1, 64, 32, 32, 32]           1,792
            Conv3d-3      [-1, 128, 32, 32, 32]         221,312
            Conv3d-4      [-1, 128, 32, 32, 32]         221,312
            Linear-5                 [-1, 3072]         789,504
            Linear-6                 [-1, 3072]         789,504
      ExampleModel-7              [-1, 1024, 3]               0
      ExampleModel-8              [-1, 1024, 3]               0
Total params: 2,025,216
Trainable params: 2,025,216
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.12
Forward/backward pass size (MB): 96.09
Params size (MB): 7.73
Estimated Total Size (MB): 103.94
----------------------------------------------------------------

Model Parameters:
m

In [11]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

# Define a loss function using Chamfer Distance
class ChamferDistance(nn.Module):
    def forward(self, points1, points2):
        points1 = points1.float()
        points2 = points2.float()
        batch_size, num_points, _ = points1.size()
        dist1 = torch.cdist(points1, points2)
        dist2 = torch.cdist(points2, points1)
        loss1 = dist1.min(dim=2)[0].mean(dim=1)
        loss2 = dist2.min(dim=2)[0].mean(dim=1)
        loss = (loss1 + loss2).mean()
        return loss

loss_fn = ChamferDistance()

class ExampleModel(nn.Module):
    def __init__(self):
        super(ExampleModel, self).__init__()
        self.conv1 = nn.Conv3d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.fc1 = None  # Initialize fc1 as None
        self.fc2 = nn.Linear(256, 1024 * 3)  # Adjust to output a batch of 1024 points (each with 3 coordinates)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1)
        if self.fc1 is None:
            flattened_size = x.size(1)
            self.fc1 = nn.Linear(flattened_size, 256)
            self.fc1.to(x.device)  # Ensure the new layer is on the correct device
        x = self.fc1(x)
        x = self.fc2(x)
        x = x.view(-1, 1024, 3)  # Reshape to [batch_size, num_points, 3]
        return x

# Function to transform model output to point cloud
def transform_output_to_point_cloud(output):
    return output

# Wrapper function to handle model call with device
def call_model_with_device(model, vertices, device):
    return model(vertices.to(device))

# Testing function
def test(model, test_loader, loss_fn, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.eval()
        test_loss = 0
        test_accuracy = 0
        with torch.no_grad():
            for voxel_grid in test_loader:
                voxel_grid = voxel_grid.to(device)
                voxel_grid_points = voxel_grid.view(voxel_grid.size(0), -1, 3)  # Reshape voxel_grid to [batch_size, num_points, 3]

                model_output = call_model_with_device(model, voxel_grid, device)
                outputs = transform_output_to_point_cloud(model_output)

                # Calculate loss
                loss = loss_fn(outputs, voxel_grid_points)
                test_loss += loss.item()

                # Calculate accuracy
                accuracy = calculate_accuracy(outputs, voxel_grid_points)
                test_accuracy += accuracy

            test_loss /= len(test_loader)
            test_accuracy /= len(test_loader)

        print(f'Epoch {epoch+1}/{num_epochs} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Function to calculate accuracy
def calculate_accuracy(outputs, voxel_grid_points):
    chamfer_dist = ChamferDistance()
    outputs = outputs.float()
    voxel_grid_points = voxel_grid_points.float()
    accuracy = 1 / (1 + chamfer_dist(outputs, voxel_grid_points).item())
    return accuracy

class VoxelGridDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.npz')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)
        voxel_grid = data['voxel_grid']
        voxel_grid = torch.tensor(voxel_grid, dtype=torch.float32)
        return voxel_grid

def custom_collate_fn(batch):
    voxel_grids = torch.stack(batch)
    return voxel_grids

def main():
    # Paths to your data directories
    test_dir = '/home/artificial-architecture/Desktop/TripoFT-adya/split_data/test'

    # Create dataset instance
    test_dataset = VoxelGridDataset(test_dir)

    # Create data loader with custom collate function
    batch_size = 1  # Adjust as needed
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)

    # Move model to GPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = ExampleModel().to(device)

    # Wrap the model with DataParallel to utilize multiple GPUs
    model = nn.DataParallel(model, device_ids=[0, 1, 2, 3])

    # Print the model summary
    print("Model Summary:")
    summary(model, (1, 32, 32, 32))  # Assuming the input size is (1, 32, 32, 32), adjust as needed

    # Print all model parameters
    print("\nModel Parameters:")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {param.shape}")

    # Test the model for a specified number of epochs
    test(model, test_loader, loss_fn, device, num_epochs=10)

if __name__ == "__main__":
    main()


Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 32, 32, 32]           1,792
            Conv3d-2       [-1, 64, 32, 32, 32]           1,792
            Conv3d-3      [-1, 128, 32, 32, 32]         221,312
            Conv3d-4      [-1, 128, 32, 32, 32]         221,312
            Linear-5                 [-1, 3072]         789,504
      ExampleModel-6              [-1, 1024, 3]               0
            Linear-7                 [-1, 3072]         789,504
      ExampleModel-8              [-1, 1024, 3]               0
Total params: 2,025,216
Trainable params: 2,025,216
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.12
Forward/backward pass size (MB): 96.09
Params size (MB): 7.73
Estimated Total Size (MB): 103.94
----------------------------------------------------------------

Model Parameters:
m